# MeerKAT conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.25 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_meerkat.ms")

[2024-04-11 10:32:04,003]     INFO  graphviper:  Couldn't find file metadata locally in /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox/file.download.json, checking remote API ... 


small_meerkat.ms.zip:   0%|          | 0.00/2.28M [00:00<?, ?iB/s]

[2024-04-11 10:32:05,570]     INFO  graphviper:  Extracting file: small_meerkat.ms.zip ... 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

ms_file = "small_meerkat.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_meerkat.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,"small_meerkat_ddi_0_intent_CALIBRATE_BANDPASS,...",0,"CALIBRATE_BANDPASS,CALIBRATE_FLUX",0,J1939-6342,3.265869e+09,3.276337e+09,"(74, 6, 50, 4)","[fk5, 19h39m25.03s, -63d42m45.6s]"
1,small_meerkat_ddi_0_intent_TARGET_field_id_2,0,TARGET,2,J0358-8103,3.265869e+09,3.276337e+09,"(223, 6, 50, 4)","[fk5, 3h58m31.5s, -81d03m45.7s]"
2,"small_meerkat_ddi_0_intent_CALIBRATE_PHASE,CAL...",0,"CALIBRATE_PHASE,CALIBRATE_AMPLI",1,J1619-8418,3.265869e+09,3.276337e+09,"(42, 6, 50, 4)","[fk5, 16h19m33.97s, -84d18m19.1s]"


In [12]:
ps.get(0).VISIBILITY.attrs["field_info"]["delay_direction"]  #'dims' ?

ps.get(0).coords["time"].attrs["integration_time"]  #'dims' ?

ps.get(0).VISIBILITY.attrs["field_info"]

{'code': 'T',
 'delay_direction': {'attrs': {'frame': 'FK5',
   'type': 'sky_coord',
   'units': ['rad', 'rad']},
  'data': [5.146178203219012, -1.1119958085589738],
  'dims': ''},
 'field_id': 0,
 'name': 'J1939-6342',
 'phase_direction': {'attrs': {'frame': 'FK5',
   'type': 'sky_coord',
   'units': ['rad', 'rad']},
  'data': [5.146178203219012, -1.1119958085589738],
  'dims': ''},
 'reference_direction': {'attrs': {'frame': 'FK5',
   'type': 'sky_coord',
   'units': ['rad', 'rad']},
  'data': [5.146178203219012, -1.1119958085589738],
  'dims': ''}}

In [6]:
ps.get(0).attrs["antenna_xds"]

<xarray.Dataset> Size: 516B
Dimensions:        (antenna_id: 3, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 24B 0 1 2
    mount          (antenna_id) <U6 72B dask.array<chunksize=(3,), meta=np.ndarray>
    name           (antenna_id) <U4 48B dask.array<chunksize=(3,), meta=np.ndarray>
    station        (antenna_id) <U4 48B dask.array<chunksize=(3,), meta=np.ndarray>
    type           (antenna_id) <U12 144B dask.array<chunksize=(3,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>